In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=40a8e99a4df2550b7bb163059485517497c477da0262992310dfc3034e3d71e8
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


Problema 2 - Diego Rodrgiuez - Joaquín Cabello

In [30]:
from pyspark.sql import SparkSession
import numpy as np

In [3]:
# Set-up la sesión
spark = SparkSession.builder \
    .getOrCreate()

sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

1.2 Problema 2. Siguiendo la misma estrategia de arriba, en esta pregunta tienes que desarrollar un algo-
ritmo con las funciones b´asicas de RDDs para computar los costos de los caminos m´as cortos partiendo de
un nodo inicial pensando en que pasas mensajes entre nodos. Al igual que en el punto anterior, la idea
es que el algoritmo que implementas se pueda ejecutar en un entorno distribuido. El output del c´odigo debe
ser cada nodo, junto al costo asociado a llegar a ese nodo, considerando que partimos de un nodo inicial i
indicado por el usuario. Adem´as, en este caso el RDD de las aristas se ve de la siguiente forma:

- edges = [(1, 2, 10), (2, 3, 3), (2, 4, 24), (3, 2, 1)]

En donde el tercer elemento indica el costo de la arista. Ahora, te dejamos una idea del algoritmo para
que entiendas a grandes rasgos los pasos.
1. Escoge el nodo inicial, este nodo tiene costo acumulado 0 y todos los dem´as tienen costo acumulado
infinito.
2. En cada iteraci´on, cada nodo comunica el costo acumulado a sus vecinos. Cada nodo recibe este costo,
sumado con el costo de atravesar la arista.
3. Para hacer merge de todos los mensajes dejamos el m´ınimo de todos los costos. As´ı, actualizamos cada
nodo con el costo m´ınimo recibido solo si es menor al costo acumulado que ya ten´ıa ese nodo.
4. Si en dos iteraciones el costo en llegar para cada nodo no cambia, entonces nos detenemos

In [6]:
def p_rdd(rdd):
  print(rdd.collect())

In [107]:

def source_shortest_path(nodos,edges, nodo_i, last_iter=None, iter=0):
  if iter == 0:
    nodos_rdd = sc.parallelize(nodos)
    edges_rdd = sc.parallelize(edges)

    edges_rdd = edges_rdd.map(lambda x: (x[0], (x[1], x[2]))).map(lambda x: (x[0], [x[1]])).reduceByKey(lambda x,y: x + y)
    nodos_cost = nodos_rdd.map(lambda x: (x, 0 if x == nodo_i else np.inf))

    n_e = nodos_cost.leftOuterJoin(edges_rdd).map(lambda x: (x[0], (x[1][0], [x[1][1]] if type(x[1][1]) != type(list()) else x[1][1])))
    last_iter = n_e.map(lambda x:x)
  else:
    n_e = last_iter.map(lambda x: x)
    nodos_rdd = nodos
    edges_rdd = edges


  # p_rdd(n_e)

  adj_list = n_e.filter(lambda x: x[1][1][0] != None).flatMap(lambda x: [(a[0], [a[1] + x[1][0]]) for a in x[1][1]  ] )\
                                                      .reduceByKey(lambda x,y: x + y).map(lambda x: (x[0], min(x[1])))

  n_e = n_e.leftOuterJoin(adj_list).map(lambda x: (x[0], (x[1][0], x[1][1] if x[1][1] != None else np.inf))).map(lambda x: (x[0], (min(x[1][0][0], x[1][1]), x[1][0][1])))





  if n_e.map(lambda x: (x[0], x[1][0])).join(last_iter.map(lambda x: (x[0], x[1][0]))).map(lambda x: max(x[1])- min(x[1]) if min(x[1]) != np.inf else 0).reduce(lambda x,y: x + y) != 0:
    return source_shortest_path(nodos_rdd,edges_rdd, nodo_i,last_iter=n_e, iter=1)
  else:
    return n_e.map(lambda x: (x[0], x[1][0]))



In [18]:
nodes = [1, 2, 3, 4]
edges = [(1, 2, 10), (2, 3, 3), (2, 4, 24), (3, 2, 1)]

nodo_inicial = 1
ssp = source_shortest_path(nodes,edges, nodo_inicial)
print('Inicial')
print(nodes)
print(edges)
print('Final partimos del', nodo_inicial)
print(ssp.collect())

source_shortest_path(nodes,edges, 1).collect()

In [50]:
nodes = [1,2,3,4,5,6]
edges = [
    (1, 2, 3), (1, 3, 5), (2, 3, 2), (2, 4, 4), (3, 2, 1),
    (3, 4, 2), (3, 5, 6), (4, 5, 3), (4, 6, 7), (5, 6, 2), (6, 1, 8)
]

nodo_inicial = 1
ssp_0 = source_shortest_path(nodes,edges, nodo_inicial)
print('Inicial')
print('Nodos:',nodes)
print('Edges:',edges)
print('Final partimos del', nodo_inicial)
print(ssp_0.collect())

Ahora realizamos con la estrategia general para que funcione en Grafos:

In [110]:
def source_shortest_path_general(grafo, nodo_i, nodos=None, edges=None, last_iter=None, iter=0):

  # ------ Preparacion de los nodos ---- #
  if iter == 0:
    nodos_rdd = sc.parallelize(grafo.nodes)
    edges_rdd = sc.parallelize(grafo.edges.data())

    edges_rdd = edges_rdd.map(lambda x: (x[0], x[1], x[2]['weight'])).map(lambda x: (x[0], (x[1], x[2]))).map(lambda x: (x[0], [x[1]])).reduceByKey(lambda x,y: x + y)
    nodos_cost = nodos_rdd.map(lambda x: (x, 0 if x == nodo_i else np.inf))

    n_e = nodos_cost.leftOuterJoin(edges_rdd).map(lambda x: (x[0], (x[1][0], [x[1][1]] if type(x[1][1]) != type(list()) else x[1][1])))
    last_iter = n_e.map(lambda x:x)
  else:
    n_e = last_iter.map(lambda x: x)
    nodos_rdd = nodos
    edges_rdd = edges
  #---------------------------------------#

  # ------- Pasamos mensajes entre nodos -----#
  adj_list = n_e.filter(lambda x: x[1][1][0] != None).flatMap(lambda x: [(a[0], [a[1] + x[1][0]]) for a in x[1][1]  ] )\
                                                      .reduceByKey(lambda x,y: x + y).map(lambda x: (x[0], min(x[1])))
  # ------------------------------------------#

  # -------- Realizamos el merge -------------#
  n_e = n_e.leftOuterJoin(adj_list).map(lambda x: (x[0], (x[1][0], x[1][1] if x[1][1] != None else np.inf))).map(lambda x: (x[0], (min(x[1][0][0], x[1][1]), x[1][0][1])))

  # -------- Condicion de termino ------------#
  if n_e.map(lambda x: (x[0], x[1][0])).join(last_iter.map(lambda x: (x[0], x[1][0]))).map(lambda x: max(x[1])- min(x[1]) if min(x[1]) != np.inf else 0).reduce(lambda x,y: x + y) != 0:
    return source_shortest_path_general(grafo, nodo_i,nodos=nodos_rdd,edges=edges_rdd,last_iter=n_e, iter=1) # Actualización de nodos
  else:
    return n_e.map(lambda x: (x[0], x[1][0])) # Actualización de nodos



Importamos la información del grafo de Cora, exteraido de la actividad 12:

In [82]:
import pandas as pd
import networkx as nx
import numpy as np
from math import sqrt
from random import randint

### Información de las aristas del grafo de cora

cora_cites_path = '/content/drive/MyDrive/Universidad/Procesamiento de datos masivos/Actividades/12 - Actividad Evaluada Grafos/cora/cora/cora.cites'
cora_content_path = '/content/drive/MyDrive/Universidad/Procesamiento de datos masivos/Actividades/12 - Actividad Evaluada Grafos/cora/cora/cora.content'

### Información de las aristas del grafo de cora

citas = pd.read_csv(cora_cites_path,sep="\t",
    header=None,
    names=["target", "source"])

### Metemos esto a networkx

G = nx.from_pandas_edgelist(citas, source="source", target="target",create_using=nx.DiGraph())

### Ahora levantamos la información de cada paper. Solo vamos a usar el "subject"

### Información de los nodos de cora

column_names = ["paper_id"] + [f"word_{idx}" for idx in range(1433)] + ["subject"]
papers = pd.read_csv(
    cora_content_path, sep="\t", names=column_names,
)
subjects = papers[["paper_id","subject"]]

### Agregamos la información a nuestro grafo G de esta forma:

subjects_dict = dict(zip(subjects['paper_id'], subjects['subject']))
nx.set_node_attributes(G, subjects_dict, 'subject')

In [84]:
# Le añadimos costos aleatorios, ya que, no posee.
total_nodos = len(G.nodes)
for go, rec in G.edges:
  G[go][rec]['weight'] = randint(0,total_nodos)

In [85]:
G.edges.data()

OutEdgeDataView([(1033, 35, {'weight': 1603}), (1033, 41714, {'weight': 1308}), (1033, 45605, {'weight': 1493}), (35, 82920, {'weight': 785}), (35, 210871, {'weight': 2030}), (35, 210872, {'weight': 830}), (103482, 35, {'weight': 1732}), (103515, 35, {'weight': 1302}), (103515, 8865, {'weight': 2592}), (1050679, 35, {'weight': 1093}), (1050679, 3229, {'weight': 1496}), (1050679, 35797, {'weight': 1074}), (1050679, 427606, {'weight': 1750}), (1103960, 35, {'weight': 1331}), (1103960, 3229, {'weight': 1467}), (1103960, 33895, {'weight': 2072}), (1103960, 33904, {'weight': 529}), (1103960, 33907, {'weight': 2301}), (1103985, 35, {'weight': 1106}), (1103985, 1688, {'weight': 1925}), (1109199, 35, {'weight': 1327}), (1109199, 6214, {'weight': 2604}), (1109199, 148170, {'weight': 967}), (1112911, 35, {'weight': 2024}), (1112911, 887, {'weight': 1218}), (1112911, 22229, {'weight': 2474}), (1112911, 144701, {'weight': 2668}), (1113438, 35, {'weight': 60}), (1113438, 35061, {'weight': 1240}), (

In [115]:
nodo_inicial = 33904
ssp_general = source_shortest_path_general(G, nodo_inicial)
print('Inicial')
print(nodes)
print(edges)
print('Final partimos del', nodo_inicial)
print(ssp_general.collect())

Inicial
[1, 2, 3, 4]
[(1, 2, 10), (2, 3, 3), (2, 4, 24), (3, 2, 1)]
Final partimos del 33904
[(1153280, inf), (1154176, inf), (273152, 3366), (128, inf), (1118848, inf), (1152896, inf), (1117184, inf), (469504, inf), (8832, inf), (235776, inf), (6784, inf), (7296, inf), (105856, inf), (1153024, inf), (37888, inf), (41216, inf), (48768, inf), (217984, inf), (61312, inf), (67584, inf), (68224, inf), (689152, inf), (1122304, inf), (228992, inf), (851968, inf), (376704, inf), (1132416, inf), (576257, inf), (7297, inf), (763009, inf), (255233, inf), (1131137, inf), (263553, inf), (6913, inf), (1155073, inf), (7041, inf), (8961, inf), (653441, inf), (1118209, inf), (1121537, inf), (1104769, inf), (31105, inf), (31489, inf), (593921, inf), (294145, inf), (212097, inf), (646913, inf), (28290, inf), (130, inf), (1026, inf), (310530, inf), (578306, inf), (77826, inf), (1132418, inf), (6786, inf), (315266, inf), (763010, inf), (28674, inf), (249858, inf), (1102850, inf), (523394, inf), (649730, i

Probamos con el ejemplo del pdf:

In [114]:
nodes = [1, 2, 3, 4]
edges = [(1, 2, 10), (2, 3, 3), (2, 4, 24), (3, 2, 1)]

G_0 = nx.DiGraph()
G_0.add_nodes_from(nodes)
for e_0,e_1,c in edges:
  G_0.add_edge(e_0,e_1, weight=c)

nodo_inicial = 1
ssp_general_0 = source_shortest_path_general(G_0, nodo_inicial)
print('Inicial')
print(nodes)
print(edges)
print('Final partimos del', nodo_inicial)
print(ssp_general_0.collect())

Inicial
[1, 2, 3, 4]
[(1, 2, 10), (2, 3, 3), (2, 4, 24), (3, 2, 1)]
Final partimos del 1
[(1, 0), (2, 10), (3, 13), (4, 34)]
